# LambdaFace

In [ ]:
from distutils.command.clean import clean
from email.mime import image
import json
import os
import logging
import boto3
import urllib
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor
import base64



def lambda_handler():
    # TODO implement

    # logger = logging.getLogger()
    # logger.setLevel(logging.INFO)

    # bucket = event['Records'][0]['s3']['bucket']['name']
    # key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')

    bucket='analitica-rekognition-bucket'
    key='caras.jpg'
   
    print('Llamada de crop')

    dictionary, num, image= detect_faces(bucket,key)

    print('\n Resultado final: \n')
    print(dictionary)
    print('número de personas: ' + '{0:.0f}'.format(num) + '\n')

    print('Llamada de crop')
    listaimg=cropFace(image,dictionary,num,key)

    for image in listaimg:

        image_data = Image.open(io.BytesIO(image))
        var = 'JPEG'
        # image_data.save(r'C:\Users\user\Desktop',var)
        image_data.show()

        #Se busca si la persona esta en una collection
        #Se recibe un booleano donde true indica que si pertence y false que no pertenece
        faceids=search_faces(image)

        #Se actualiza atributo (Status) en la base de datos en DynamoDB
        #de acuerdo al resultado de la función search_faces para cada una de las coincidencias en la collection
        for faceid in faceids:
            updateItemDB(faceid)



   

def detect_faces(bucket,key):

    client=boto3.client('rekognition', 'us-east-1')


    print(bucket)
    print(key)
    
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': key}},
        Attributes=['ALL'])

    print('Detected faces for ' + key)   

    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
            + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        # print('Here are the other attributes:')
        # print(json.dumps(faceDetail, indent=4, sort_keys=True))

        # Access predictions for individual face details and print them
        print("Gender: " + str(faceDetail['Gender']))
        # print("Smile: " + str(faceDetail['Smile']))
        # print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        # print("Emotions: " + str(faceDetail['Emotions'][0]))


    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket,key)
    s3_response = s3_object.get()

    

    stream = io.BytesIO(s3_response['Body'].read())

    print('Hasta aquí se ejecura sin PIL')
    image=Image.open(stream)

    imgWidth, imgHeight = image.size  

    print('Se va a imprimir imagen')

    # draw = ImageDraw.Draw(image)  
                
    dict={}


    count=1

    # calculate and display bounding boxes for each detected face       
    # print('Bounding boxes for ' + key)    
    for faceDetail in response['FaceDetails']:
    

        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']

        dict['cara'+'{0:.0f}'.format(count)]=[left,top,left+width,top+height]
                
        count=count+1


        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )
        # draw.line(points, fill='#00d400', width=2)

    print('número de caras')
    numCaras=len(response['FaceDetails'])


    # image.show()

    return dict, numCaras, image


def cropFace(image,dict,numCaras,key):

    client=boto3.client('s3')

    key=key[0:len(key)-4]

    listaimg=[]

    for i in range(1,numCaras+1):
        
        nombre=key+"{0:.0f}".format(i)

        dimensiones=dict['cara'+"{0:.0f}".format(i)]

        dim=(int(dimensiones[0]),int(dimensiones[1]),int(dimensiones[2]),int(dimensiones[3]))
        imagecrop=image.crop(dim)


        img_byte_arr = io.BytesIO()


        imagecrop.save(img_byte_arr, format="JPEG")

        # img_str = base64.b64encode(img_byte_arr.getvalue())

        # imagecrop.save(img_byte_arr, format='JPEG')

        image_file_size = img_byte_arr.tell()

        print('Tamaño (bytes)')
        print(image_file_size)

        img_byte_arr = img_byte_arr.getvalue()

        listaimg.append(img_byte_arr)

    #     print(nombre+'.jpeg')

    #     # client.upload_fileobj(img_str.read(), 'prueba-rekognition-analitica', nombre+'.jpeg')

    #     client.put_object(
    # Body=img_byte_arr,
    # Bucket='prueba-rekognition-analitica',
    # Key=nombre+'.jpeg',

# )

    print(len(listaimg))

    return listaimg

#Se define función para buscar caras en una imagen y relacionar con una collection
#Recibe el nombre del bucket y de la imagen a analizar 
#Retorna lista con los FaceId de las coincidencias en la colección
def search_faces(image):

    #Cliente representando servicio de rekognition
    client=boto3.client('rekognition', 'us-east-1')


    collectionId = 'CollectionAnalitica' #Nombre de la colección
    threshold = 80 #Umbral para similaridad entre caras
    maxFaces = 100 #Número máximo de caras que quiere reconocer de la colección
    
    #Función del SDK (boto3) de python para buscar coincidencia con caras de una colección


    response=client.search_faces_by_image(CollectionId=collectionId,
                                    Image={'Bytes': image},
                                    FaceMatchThreshold=threshold,
                                     MaxFaces=maxFaces)


    faceMatches = response['FaceMatches']

    #Lista con el FaceId de la cara de coincidencia en la colección
    listface=[]


    for match in faceMatches:
        print('FaceId:' + match['Face']['FaceId'])
        print('ImageId:' + match['Face']['ImageId'])
        print('Similarity: ' + "{:.2f}".format(match['Similarity']) + "%")
        print('Confidence: ' + str(match['Face']['Confidence']))

        #Si la similaridad entre coincidencia es mayor a 80% se agrega el FaceId a la lista listface
        if match['Similarity'] > 80.0:
            listface.append( match['Face']['FaceId'])

    return listface


#Se define función para actualizar un item de la base de datos de dynamodb
#Recibe el FaceId del item a actualizar
def updateItemDB(FaceId):

    #Cliente representando servicio dynamodb
    client = boto3.client('dynamodb')

    try:

        #Se actualiza la base de datos cambiando el atributo status con el valor booleano True
        response = client.update_item(
            TableName='rekognition-machine-analitic',
            Key={
                'FaceID': {
                    'S': FaceId
                }
            },
            AttributeUpdates={
                'Status': {
                    'Value': {
                        'S': 'OK'
                    },
                    'Action': 'PUT'
                }
            },


        )   

        print('Actualizó DB')

    except Exception as msg:

        print(f"Oops, no se pudo actualizar el item: {msg}")

lambda_handler()

SNS

In [10]:
import boto3
import json

notification = "Here is the SNS notification for Lambda function tutorial."
client = boto3.client('sns')

response = client.publish (
    TargetArn = "arn:aws:sns:us-east-1:533886999211:SNSPrueba",
    Message = json.dumps({'default': notification}),
    MessageStructure = 'json'
)



In [11]:
import boto3
import json

client = boto3.client('sns')

snsArn = 'arn:aws:sns:us-east-1:533886999211:FaceSNS'
message = "This is a test notification."

response = client.publish(
    TopicArn = snsArn,
    Message = message ,
    Subject='Hello'
)

In [ ]:
response = client.get_item(
    TableName='string',
    Key={
        'string': {
            'S': 'string',
            'N': 'string',
            'B': b'bytes',
            'SS': [
                'string',
            ],
            'NS': [
                'string',
            ],
            'BS': [
                b'bytes',
            ],
            'M': {
                'string': {'... recursive ...'}
            },
            'L': [
                {'... recursive ...'},
            ],
            'NULL': True|False,
            'BOOL': True|False
        }
    },
    AttributesToGet=[
        'string',
    ],
    ConsistentRead=True|False,
    ReturnConsumedCapacity='INDEXES'|'TOTAL'|'NONE',
    ProjectionExpression='string',
    ExpressionAttributeNames={
        'string': 'string'
    }
)

In [11]:
import cv2
cap = cv2.VideoCapture("rtsp://192.168.1.16:554/live1s1.sdp",cv2.CAP_FFMPEG)

 #Tamaño de la imagen transmitida
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(width)
print(height)

#Se ajusta tamaño de la recepción de acuerdo al tamaño de
#la imagen del video transmitido.
cap.set(3, width)
cap.set(4, height)

if not cap.isOpened():
    print('Cannot open RTSP stream')
    exit(-1)

while True:
    _, frame = cap.read()
    cv2.imshow('RTSP stream', frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

1920.0
1080.0


In [10]:
import base64
with open("/home/analitica2/analitica2/20220921_155031M.jpg", "rb") as img_file:
    if img_file.read():
        print("leyo bien")
        print((base64.b64encode(img_file.read()).decode('utf-8')))

leyo bien



In [9]:
import pika
import json
#método para públicar
def publish(message, queue, mqHost):

    #Se establece conexión con el servidor
    connection = pika.BlockingConnection(pika.ConnectionParameters(mqHost))

    #Se comienza el canal de comunicación
    channel = connection.channel()

    #Se establece la cola
    channel.queue_declare(queue, passive=False, durable=False, exclusive=False, auto_delete=False, arguments=None)

    #Se envía mensaje
    channel.basic_publish("", queue, bytes(message, 'utf-8'), properties=None, mandatory=False)
    print(" [x] Sent %r" % message)

    #Se cierra la conexión
    connection.close()

data = {}
            
data["path"] = "/home/analitica2/Documentos/ftp/20220922_073004M.jpg"
mqHost = "localhost"   

#Se envía mensaje a la cola del servicio de mensajería
publish(json.dumps(data), "captured-image-queue", mqHost)

print("Se envio imagen")
        








 [x] Sent '{"path": "/home/analitica2/Documentos/ftp/20220922_073004M.jpg"}'
Se envio imagen


## Umbral Área de Bounding Box

In [5]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor

def show_faces(photo,bucket):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    # for orientation in ExifTags.TAGS.keys():
    #     if ExifTags.TAGS[orientation]=='Orientation':
    #         break
    
    # exif = image._getexif()

    # if exif[orientation] == 3:
    #     image=image.rotate(180, expand=True)
    # elif exif[orientation] == 6:
    #     image=image.rotate(270, expand=True)
    # elif exif[orientation] == 8:
    #     image=image.rotate(90, expand=True)

    #Call DetectFaces 
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        Attributes=['ALL'])

    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
                    

    # calculate and display bounding boxes for each detected face       
    print('Detected faces for ' + photo)    
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * 1 * box['Width']
        height = imgHeight * 1 * box['Height']
                

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )

        area = width*height
        print("AREA: " +str(area))

        if area > 8000:
            draw.line(points, fill='#00d400', width=2)

        # Alternatively can draw rectangle. However you can't set line width.
        #draw.rectangle([left,top, left + width, top + height], outline='#00d400') 

    image.show()

    return len(response['FaceDetails'])

def main():
    # bucket="bucket-captures"
    # photo="em.jpg"

    bucket="bucket-captures"
    photo="2022101394333138015.jpg"



    faces_count=show_faces(photo,bucket)
    print("faces detected: " + str(faces_count))


if __name__ == "__main__":
    main()

Detected faces for 2022101394333138015.jpg
The detected face is between 43 and 51 years old
Left: 471
Top: 260
Face Width: 77
Face Height: 106
AREA: 8194.353211380076
faces detected: 1


In [12]:
import boto3
import json

def detect_faces(photo, bucket):

    #Activar rekognition
    client=boto3.client('rekognition')

    #Activar detec-faces
    response = client.detect_faces(Image={'S3Object':{'Bucket':bucket,'Name':photo}},Attributes=['ALL'])

    # print('Detected faces for ' + photo)    
    for faceDetail in response['FaceDetails']:
    #     print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
    #           + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        # print('Here are the other attributes:')
        # print(json.dumps(faceDetail, indent=4, sort_keys=True))
        print(str(faceDetail["Gender"]))
        print(str(faceDetail["Pose"]))

		# # Access predictions for individual face details and print them
        # print("Gender: " + str(faceDetail['Gender']))
        # print("Smile: " + str(faceDetail['Smile']))
        # print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        # print("Emotions: " + str(faceDetail['Emotions'][0]))

    # return len(response['FaceDetails'])
def main():
    photo='2022927133955183669.jpg'
    bucket='prueba-rekognition-analitica'
    face_count=detect_faces(photo, bucket)
    # print("Faces detected: " + str(face_count))


if __name__ == "__main__":
    main()

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the DetectFaces operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

### FACENET

In [ ]:
import torch
from facenet_pytorch import MTCNN
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Lectura de imágenes
# ==============================================================================
from PIL import Image
import cv2
import matplotlib.pyplot as plt

#imagen_1 = Image.open('/home/analitica2/Documentos/borrar/20220922_134554.jpg')
#imagen_2 = Image.open('/home/analitica2/Documentos/borrar/20220922_134552.jpg')

imagen_1 = cv2.imread('C:/Users/user/Pictures/natalia.jpg')
imagen_1 = cv2.cvtColor(imagen_1,cv2.COLOR_BGR2RGB)
imagen_2 = cv2.imread('C:/Users/user/Pictures/telemetrik.jpeg')
imagen_2 = cv2.cvtColor(imagen_2,cv2.COLOR_BGR2RGB)

# # Representación de imágenes
# # ==============================================================================
# plt.figure(figsize=(5, 4))
# plt.imshow(imagen_1)
# plt.axis('off')
# plt.show()

# plt.figure(figsize=(10, 6))
# plt.imshow(imagen_2)
# plt.axis('off')
# Detectar si se dispone de GPU cuda
# ==============================================================================
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print('Running on device: {}'.format(device))

# Detector MTCNN
# ==============================================================================
mtcnn = MTCNN(
            select_largest = True,
            min_face_size  = 15,
            thresholds     = [0.8, 0.75, 0.75],
            post_process   = False,
            image_size     = 160,
            # device         = device
        )
# Detección de bounding box y landmarks
# ==============================================================================
boxes, probs, landmarks = mtcnn.detect(imagen_1, landmarks=True)
print('Bounding boxes:', boxes)
print('Probability:', probs)
print('landmarks:', landmarks)



# Representación con matplotlib
# ==============================================================================
# En punto de origen (0,0) de una imagen es la esquina superior izquierda
box = boxes[0]
landmark = landmarks[0]
fig, ax  = plt.subplots(figsize=(5, 4))
ax.imshow(imagen_1)
ax.scatter(landmark[:, 0], landmark[:, 1], s=8, c= 'red')
rect = plt.Rectangle(
            xy     = (box[0], box[1]),
            width  = box[2] - box[0],
            height = box[3] - box[1],
            fill   = False,
            color  = 'red'
       )
ax.add_patch(rect)
ax.axis('off')
plt.show()



#pathRecuadro = "/home/analitica2/Documentos/RecuadrosPersonas/"+"12.jpg"
        
#cv2.imwrite(pathRecuadro, imagen_1)

# Detección de bounding box y landmarks
# ==============================================================================
boxes, probs, landmarks = mtcnn.detect(imagen_2, landmarks=True)

try:
    # Representación con matplotlib
    # ==============================================================================
    fig, ax = plt.subplots(figsize=(12, 7))
    ax.imshow(imagen_2)

    for box, landmark in zip(boxes, landmarks):
        ax.scatter(landmark[:, 0], landmark[:, 1], s=8, c= 'red')
        rect = plt.Rectangle(
                    xy     = (box[0], box[1]),
                    width  = box[2] - box[0],
                    height = box[3] - box[1],
                    fill   = False,
                    color  = 'red'
            )
        ax.add_patch(rect)
        
    ax.axis('off')
    plt.show()

except:
    print("No se encontraron caras")